In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import random
import re

from tqdm import tqdm
import time

import pydicom as dicom
import nibabel as nib
import SimpleITK as sitk
import monai

import torch
import torch.nn as nn
import torch.optim as optim

from monai.networks.nets import EfficientNetBN
from monai.networks.nets import ResNet
#from efficientnet_pytorch import EfficientNet

import wandb


In [2]:
SEED = 344
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # Fix the network according to random seed
    print('Finish seeding with seed {}'.format(seed))
    
seed_everything(SEED)
print('Training on device {}'.format(device))

Finish seeding with seed 344
Training on device cuda


In [3]:
dicom_tag_columns = [
    'Columns',
    'ImageOrientationPatient',
    'ImagePositionPatient',
    'InstanceNumber',
    'PatientID',
    'PatientPosition',
    'PixelSpacing',
    'RescaleIntercept',
    'RescaleSlope',
    'Rows',
    'SeriesNumber',
    'SliceThickness',
    'path',
    'WindowCenter',
    'WindowWidth'
]

train_dicom_tags = pd.read_parquet('autodl-tmp/train_dicom_tags.parquet', columns=dicom_tag_columns)
test_dicom_tags = pd.read_parquet('autodl-tmp/test_dicom_tags.parquet', columns=dicom_tag_columns)

train_series_meta = pd.read_csv('autodl-tmp/train_series_meta.csv')
test_series_meta = pd.read_csv('autodl-tmp/test_series_meta.csv')

train_csv = pd.read_csv('autodl-tmp/train.csv')

train_csv

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,9951,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3143,9960,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3144,9961,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3145,9980,1,0,1,0,1,0,0,1,0,0,0,0,1,1


In [4]:
injury_series_meta = train_series_meta.loc[train_series_meta.patient_id.isin(train_csv.loc[train_csv.any_injury == 1, "patient_id"].values)]
healthy_series_meta = train_series_meta.loc[train_series_meta.patient_id.isin(train_csv.loc[train_csv.any_injury == 0, "patient_id"].values)]

In [5]:
def raw_path_gen(patient_id, series_id, train=True):
    if(train):
        path = 'autodl-tmp/train_images_resample/'
    else:
        path = 'autodl-tmp/train_images_resample/'
    
    path += str(patient_id) + '/' + str(series_id)
    
    return path

def create_3D_scans(folder, downsample_rate=1): 
    filenames = os.listdir(folder)
    filenames = [int(filename.split('.')[0]) for filename in filenames]
    filenames = sorted(filenames)
    filenames = [str(filename) + '.dcm' for filename in filenames]
        
    volume = []
    #for filename in tqdm(filenames[::downsample_rate], position=0): 
    for filename in filenames[::downsample_rate]: 
        filepath = os.path.join(folder, filename)
        ds = dicom.dcmread(filepath)
        image = ds.pixel_array
        
        if ds.PixelRepresentation == 1:
            bit_shift = ds.BitsAllocated - ds.BitsStored
            dtype = image.dtype 
            image = (image << bit_shift).astype(dtype) >>  bit_shift
        
        # find rescale params
        if ("RescaleIntercept" in ds) and ("RescaleSlope" in ds):
            intercept = float(ds.RescaleIntercept)
            slope = float(ds.RescaleSlope)
    
        # find clipping params
        center = int(ds.WindowCenter)
        width = int(ds.WindowWidth)
        low = center - width / 2
        high = center + width / 2    
        
        
        image = (image * slope) + intercept
        image = np.clip(image, low, high)

        image = (image / np.max(image) * 255).astype(np.int16)
        image = image[::downsample_rate, ::downsample_rate]
        volume.append( image )
    
    volume = np.stack(volume, axis=0)
    return volume

def plot_image_with_seg(volume, volume_seg=[], orientation='Coronal', num_subplots=20):
    # simply copy
    if len(volume_seg) == 0:
        plot_mask = 0
    else:
        plot_mask = 1
        
    if orientation == 'Coronal':
        slices = np.linspace(0, volume.shape[2]-1, num_subplots).astype(np.int16)
        volume = volume.transpose([1, 0, 2])
        if plot_mask:
            volume_seg = volume_seg.transpose([1, 0, 2])
        
    elif orientation == 'Sagittal':
        slices = np.linspace(0, volume.shape[2]-1, num_subplots).astype(np.int16)
        volume = volume.transpose([2, 0, 1])
        if plot_mask:
            volume_seg = volume_seg.transpose([2, 0, 1])

    elif orientation == 'Axial':
        slices = np.linspace(0, volume.shape[0]-1, num_subplots).astype(np.int16)
           
    rows = np.max( [np.floor(np.sqrt(num_subplots)).astype(int) - 2, 1])
    cols = np.ceil(num_subplots/rows).astype(int)
    
    fig, ax = plt.subplots(rows, cols, figsize=(cols * 2, rows * 4))
    fig.tight_layout(h_pad=0.01, w_pad=0)
    
    ax = ax.ravel()
    for this_ax in ax:
        this_ax.axis('off')

    for counter, this_slice in enumerate( slices ):
        plt.sca(ax[counter])
        
        image = volume[this_slice, :, :]
        plt.imshow(image, cmap='gray')
        
        if plot_mask:
            mask = np.where(volume_seg[this_slice, :, :], volume_seg[this_slice, :, :], np.nan)
            plt.imshow(mask, cmap='Set1', alpha=0.5)
            
def load_nii(patient_id, series_id, root='autodl-tmp/train_images_resample/'):
    path = root + str(patient_id) + '/' + str(series_id) + '.nii.gz'
    img = sitk.ReadImage(path)
    img = sitk.GetArrayFromImage(img)
    
    # img = nib.load(path)
    # img = img.get_fdata().transpose(2, 1, 0)
    
    return img

In [6]:
# t = monai.transforms.Compose([#monai.transforms.NormalizeIntensity(),
#                                     monai.transforms.RandRotate(range_x=3.14 / 12, prob=1),
#                                     monai.transforms.SpatialPad(spatial_size=(176, 140, 140), mode="edge"),
#                                     monai.transforms.RandSpatialCrop(roi_size=(160, 128, 128), random_size=False),
#                                 ])
# img_t = t(np.expand_dims(img_a[::2, ::2, ::2], 0)).numpy().squeeze(0)
# img_t[:, 0, :]
# plot_image_with_seg(img_t, orientation='Axial', num_subplots=7)
# plot_image_with_seg(img_t, orientation='Sagittal', num_subplots=7)

In [7]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

ds = 2

class CTDataset(Dataset):
    def __init__(self, root='autodl-tmp/train_images_resample/', augmentation=False, meta=train_series_meta, device='cpu'):
        self.device = device
        self.series_meta = meta
        self.root = root
        self.t = monai.transforms.Compose([monai.transforms.RandZoom(prob=0.5, min_zoom=0.9, max_zoom=1.1),
                                           monai.transforms.RandRotate(range_x=3.14 / 24, prob=0.5),
                                           monai.transforms.SpatialPad(spatial_size=(352//ds, 280//ds, 280//ds), mode="edge"),
                                           monai.transforms.RandSpatialCrop(roi_size=(320//ds, 256//ds, 256//ds), random_size=False),
                                           monai.transforms.NormalizeIntensity(divisor = 400)
                                ])
        self.t_val = monai.transforms.Compose([monai.transforms.NormalizeIntensity(divisor = 400)])
        
        self.aug = augmentation
        
    def __len__(self):
        #return 1100
        return len(self.series_meta)
    
    def __getitem__(self, idx):

        patient_id, series_id = self.series_meta.loc[idx, ["patient_id", "series_id"]].astype('int')
        img_a = load_nii(patient_id, series_id, self.root).astype('float32')
        #img_t = torch.from_numpy(img_a).unsqueeze(0)
        #img_t = torch.from_numpy(img_a[::2, ::2, ::2]).unsqueeze(0)
        if(self.aug):
            img_t = self.t(np.expand_dims(img_a[::ds, ::ds, ::ds], 0))
        else:
            #img_t = torch.from_numpy(img_a[::ds, ::ds, ::ds]).unsqueeze(0)
            img_t = self.t_val(np.expand_dims(img_a[::ds, ::ds, ::ds], 0))
        label_columns = [
            'bowel_healthy',
            'bowel_injury',
            'extravasation_healthy',
            'extravasation_injury',
            'kidney_healthy',
            'kidney_low',
            'kidney_high',
            'liver_healthy',
            'liver_low',
            'liver_high',
            'spleen_healthy',
            'spleen_low',
            'spleen_high',
            #'any_injury'
        ]
        label_a = train_csv.loc[train_csv.patient_id == patient_id, label_columns].values[0].astype('float32')
        label_t = torch.from_numpy(label_a)
        return img_t, label_t

In [8]:
train_meta = train_series_meta[-3600:].reset_index()
val_meta = train_series_meta[0:-3600].reset_index()

# train_meta = injury_series_meta[-800:].reset_index()
# val_meta = injury_series_meta[0:-800].reset_index()

train_ds = CTDataset(meta = train_meta, augmentation=True)
train_dl = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=8)

val_ds = CTDataset(meta = val_meta, augmentation=False)
val_dl = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=8)

In [9]:
class EffNet(nn.Module):
    def __init__(self, ch_out=9):
        super(EffNet, self).__init__()
        #self.conv_in = nn.Conv3d(1, 3, kernel_size=5, padding=2, stride=2)
        self.net = EfficientNetBN("efficientnet-b0", pretrained=False, progress=False, spatial_dims=3, in_channels=1, num_classes=ch_out,)
    def forward(self, x):
        #x = self.conv_in(x)
        #return torch.sigmoid(self.net(x))
        return self.net(x)

In [10]:
print(EffNet().net._fc)

Linear(in_features=1280, out_features=9, bias=True)


In [11]:
import sklearn.metrics

def transform_9class(label_in):
    label_out = [1 - label_in[0],
                   label_in[0],
                    1- label_in[1],
                   label_in[1],
                   (1 - label_in[2]) * (1 - label_in[3]),
                   label_in[2],
                   label_in[3],
                   (1 - label_in[4]) * (1 - label_in[5]),
                    label_in[4],
                   label_in[5],
                   (1 - label_in[6]) * (1 - label_in[7]),
                   label_in[6],
                   label_in[7]]
    return label_out

def transform_13class(label_in):
    label_out = label_in
    return label_out.tolist()


def loss_metrics(metrics, transform):
    preds = [transform(x) for x in metrics["predict"]]
    targets = [transform(x) for x in metrics["label"]]
    targets_any_injury = metrics["label"][:, -1]
    
    loss_list = []
    
    print("F1 score: ", sklearn.metrics.f1_score(metrics["label"], np.around(metrics["predict"]), average=None, zero_division=0.0))
    print("AUC score: ", sklearn.metrics.roc_auc_score(metrics["label"], metrics["predict"], average=None))
    
    for i in range(0, len(preds)):
        predict = preds[i]
        target = targets[i]
        
        label_pred = np.zeros(14)
        label_pred[0] = predict[0] / (predict[0] + predict[1])
        label_pred[1] = predict[1] / (predict[0] + predict[1])
        label_pred[2] = predict[2] / (predict[2] + predict[3])
        label_pred[3] = predict[3] / (predict[2] + predict[3])
        label_pred[4] = predict[4] / (predict[4] + predict[5] + predict[6])
        label_pred[5] = predict[5] / (predict[4] + predict[5] + predict[6])
        label_pred[6] = predict[6] / (predict[4] + predict[5] + predict[6])
        label_pred[7] = predict[7] / (predict[7] + predict[8] + predict[9])
        label_pred[8] = predict[8] / (predict[7] + predict[8] + predict[9])
        label_pred[9] = predict[9] / (predict[7] + predict[8] + predict[9])
        label_pred[10] = predict[10] / (predict[10] + predict[11] + predict[12])
        label_pred[11] = predict[11] / (predict[10] + predict[11] + predict[12])
        label_pred[12] = predict[12] / (predict[10] + predict[11] + predict[12])
        label_pred[13] = max([1 - label_pred[x] for x in [0, 2, 4, 7, 10]])
        
        targets_any_injury = max([1 - target[x] for x in [0, 2, 4, 7, 10]])
        
        target.append(targets_any_injury)
        label_target = np.array(target)
        
        weight = np.array([1, 2, 1, 6, 1, 2, 4, 1, 2, 4, 1, 2, 4, 6])
        
        loss_list.append(sklearn.metrics.log_loss(
            y_true=label_target,
            y_pred=label_pred,
            sample_weight=weight))
    #print("Weighted Loss: " + np.mean(loss_list))
    
    return np.mean(loss_list)
        
    
    
    #print(np.array(preds).shape)
    

In [12]:
# torch.cuda.empty_cache()

# net = EffNet(ch_out=8).to(device)
# #net.load_state_dict(torch.load('EffNet3D/test_metrics.pt'))

# imgs, labels = next(iter(val_dl))

# outputs = net(imgs.to(device))

In [13]:
# print(outputs)
# print(labels)

# metrics = {"predict": outputs.to('cpu').detach().numpy(),
#           "label": labels.to('cpu').detach().numpy()}

# loss_metrics(metrics, transform_9class)


In [14]:
import copy

def TrainClassifer(model,trn_dl,val_dl,optimizer, project, name, suffix, scheduler=None,
                   n_eopchs=20, device='cpu'):
 
    #loss_fn = nn.BCELoss(weight=torch.Tensor([1, 6, 1, 6, 1, 4, 8, 1, 4, 8, 1, 4, 8]).to(device))
    #loss_fn = nn.BCELoss()
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([1, 30, 1, 15, 1, 30, 30, 1, 18, 18, 1, 18, 18]).to(device))
    model.to(device)
    best_model = copy.deepcopy(model)
    bestweight_model = copy.deepcopy(model)
    best_val = 999.0
    best_weightloss = 999.0
    metrics = {'predict': [], 'label' : []}
    PATH_MODEL2 = project + '/' + name + '/' + suffix + '.pt'
    wandb.init(name=name, 
               project=project)

    for epoch in range(1, n_eopchs + 1):
        loss_train = 0.0
        model.train()
        for imgs, labels in tqdm(trn_dl, position=0):
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            #outputs = model(imgs.unsqueeze(1))
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        torch.cuda.empty_cache()
        
        loss_val = 0.0
        correct_val = 0.0
        model.eval()
        
        for imgs, labels in tqdm(val_dl):
            imgs = imgs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = model(imgs)
                #outputs = model(imgs.unsqueeze(1))
                loss = loss_fn(outputs, labels)
                loss_val += loss.item()
                outputs = torch.sigmoid(outputs)
                metrics['predict'].extend((outputs.to('cpu').detach().numpy()).tolist())
                metrics['label'].extend((labels.to('cpu').detach().numpy()).tolist())
        
        metrics['predict'] = np.array(metrics['predict'])
        #metrics['predict'] = np.array([[0.5, 0.5, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333]]*len(metrics['label']))
        metrics['label'] = np.array(metrics['label'])
        weighted_loss = loss_metrics(metrics, transform_13class)
        metrics = {'predict': [], 'label' : []}
        
        torch.cuda.empty_cache()
        
        if (weighted_loss) < best_weightloss:
            best_weightloss = weighted_loss
            torch.save(model.state_dict(), 'model_tmp.pt')
            bestweight_model.load_state_dict(torch.load('model_tmp.pt'))
            
        if loss_val / len(val_dl) < best_val:
            best_val = loss_val / len(val_dl)
            #torch.save(model.state_dict(), 'model_tmp.pt')
            #best_model.load_state_dict(torch.load('model_tmp.pt'))
            
            
        if scheduler != None:
            scheduler.step()

        print('{} Eopch {}, Training Loss {}, Val Loss {}, Weighted Loss {}'.format(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()),
                                                                  epoch, loss_train / len(trn_dl), loss_val / len(val_dl),
                                                                                     weighted_loss))
        
        
        
        wandb.log({'training loss': loss_train / len(trn_dl),
                  'val loss': loss_val / len(val_dl),
                  'weighted loss': weighted_loss})
    #torch.save(best_model.state_dict(), PATH_MODEL)
    torch.save(bestweight_model.state_dict(), PATH_MODEL2)
    print('Finish training: best_val:{}, best_weighted loss:{}'.format(best_val, best_weightloss))
    wandb.finish()

In [15]:
import os

project = "Effnet3D-nophase-4fold"
name = "Effnet3D-nophase_4fold"
# backbone = "tf_efficientnetv2_s.in21k_ft_in1k"
# backbone = 'convnextv2_nano.fcmae_ft_in22k_in1k_384'
n_eopchs = 30

p = project + '/' + name
if not os.path.exists(p):
    os.mkdir(p)

In [16]:
# 4711 // 4 = 1178

train_df_fold1 = train_series_meta[1178:].reset_index(drop=True)
train_df_fold2 = pd.concat([train_series_meta[0:1178], train_series_meta[2356:]], axis=0).reset_index(drop=True)
train_df_fold3 = pd.concat([train_series_meta[0:2356], train_series_meta[3534:]], axis=0).reset_index(drop=True)
train_df_fold4 = train_series_meta[:3534].reset_index()

val_df_fold1 = train_series_meta[0:1178].reset_index(drop=True)
val_df_fold2 = train_series_meta[1178:2356].reset_index(drop=True)
val_df_fold3 = train_series_meta[2356:3534].reset_index(drop=True)
val_df_fold4 = train_series_meta[3534:].reset_index(drop=True)

train_ds_fold1 = CTDataset(meta = train_df_fold1, augmentation=True)
train_dl_fold1 = DataLoader(train_ds_fold1, batch_size=4, shuffle=True, num_workers=8)
val_ds_fold1 = CTDataset(meta = val_df_fold1, augmentation=False)
val_dl_fold1 = DataLoader(val_ds_fold1, batch_size=4, shuffle=False, num_workers=8)

train_ds_fold2 = CTDataset(meta = train_df_fold2, augmentation=True)
train_dl_fold2 = DataLoader(train_ds_fold2, batch_size=4, shuffle=True, num_workers=8)
val_ds_fold2 = CTDataset(meta = val_df_fold2, augmentation=False)
val_dl_fold2 = DataLoader(val_ds_fold2, batch_size=4, shuffle=False, num_workers=8)

train_ds_fold3 = CTDataset(meta = train_df_fold3, augmentation=True)
train_dl_fold3 = DataLoader(train_ds_fold3, batch_size=4, shuffle=True, num_workers=8)
val_ds_fold3 = CTDataset(meta = val_df_fold3, augmentation=False)
val_dl_fold3 = DataLoader(val_ds_fold3, batch_size=4, shuffle=False, num_workers=8)

train_ds_fold4 = CTDataset(meta = train_df_fold4, augmentation=True)
train_dl_fold4 = DataLoader(train_ds_fold4, batch_size=4, shuffle=True, num_workers=8)
val_ds_fold4 = CTDataset(meta = val_df_fold4, augmentation=False)
val_dl_fold4 = DataLoader(val_ds_fold4, batch_size=4, shuffle=False, num_workers=8)

In [ ]:
net = EffNet(ch_out=13).to(device)
optimizer = optim.AdamW(net.parameters(), lr=1e-4)
TrainClassifer(model=net,trn_dl=train_dl_fold1,val_dl=val_dl_fold1,optimizer=optimizer, 
               project=project, name=name, suffix='fold1', scheduler=None, n_eopchs=n_eopchs, device=device)

net = EffNet(ch_out=13).to(device)
optimizer = optim.AdamW(net.parameters(), lr=1e-4)
TrainClassifer(model=net,trn_dl=train_dl_fold2,val_dl=val_dl_fold2,optimizer=optimizer, 
               project=project, name=name, suffix='fold2', scheduler=None, n_eopchs=n_eopchs, device=device)

net = EffNet(ch_out=13).to(device)
optimizer = optim.AdamW(net.parameters(), lr=1e-4)
TrainClassifer(model=net,trn_dl=train_dl_fold3,val_dl=val_dl_fold3,optimizer=optimizer, 
               project=project, name=name, suffix='fold3', scheduler=None, n_eopchs=n_eopchs, device=device)

net = EffNet(ch_out=13).to(device)
optimizer = optim.AdamW(net.parameters(), lr=1e-4)
TrainClassifer(model=net,trn_dl=train_dl_fold4,val_dl=val_dl_fold4,optimizer=optimizer, 
               project=project, name=name, suffix='fold4', scheduler=None, n_eopchs=n_eopchs, device=device)

wandb: Currently logged in as: northm (rsna2023). Use `wandb login --relogin` to force relogin


100%|██████████| 295/295 [01:18<00:00,  3.74it/s]


F1 score:  [0.9875376  0.04210526 0.96621325 0.11751663 0.96621325 0.08594449
 0.04571429 0.94491715 0.14686469 0.         0.93441882 0.12099644
 0.06349206]
AUC score:  [0.64376819 0.45472825 0.56970641 0.49412577 0.51819479 0.5773576
 0.51208075 0.58061881 0.4419408  0.50643738 0.5401175  0.52088693
 0.60395051]
2023-10-05 08:49:40 Eopch 1, Training Loss 1.127587669406811, Val Loss 0.9210496858014898, Weighted Loss 0.4421842670442662


100%|██████████| 295/295 [01:20<00:00,  3.67it/s]


F1 score:  [0.9875376  0.04195804 0.96621325 0.05150215 0.96621325 0.08119304
 0.         0.94491715 0.15384615 0.         0.93393665 0.13916947
 0.1010274 ]
AUC score:  [0.48592479 0.45256745 0.52813263 0.50926548 0.52611557 0.46846586
 0.4315528  0.55677956 0.5982276  0.60538399 0.52814367 0.50126473
 0.55870602]
2023-10-05 08:56:07 Eopch 2, Training Loss 0.9480297087409377, Val Loss 0.9277353599920111, Weighted Loss 0.4449278148673551


...

 81%|████████  | 716/884 [03:59<00:52,  3.17it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████| 295/295 [01:07<00:00,  4.39it/s]


F1 score:  [0.98271392 0.10810811 0.97025372 0.17333333 0.97070398 0.07835455
 0.05749487 0.94284408 0.17727273 0.12030075 0.93792478 0.2
 0.32411067]
AUC score:  [0.74085312 0.78120053 0.66740704 0.67238636 0.64714267 0.65074758
 0.6310789  0.76950654 0.73014421 0.85730028 0.82127362 0.72625718
 0.88971949]
2023-10-05 20:36:38 Eopch 21, Training Loss 0.7148530329177283, Val Loss 0.8106171843359026, Weighted Loss 0.399279207403324


100%|██████████| 295/295 [01:16<00:00,  3.86it/s]


F1 score:  [0.98271392 0.171875   0.97025372 0.14249364 0.97070398 0.09836066
 0.06708595 0.9532294  0.17980296 0.13333333 0.93150685 0.17079208
 0.22810591]
AUC score:  [0.78922603 0.7848285  0.65859545 0.6551875  0.67965578 0.69085312
 0.67980676 0.75310502 0.70877445 0.83746556 0.81452295 0.70837656
 0.89458371]
2023-10-05 20:42:43 Eopch 22, Training Loss 0.7014521228073679, Val Loss 0.7844175901958498, Weighted Loss 0.41001221080428407


 18%|█▊        | 156/884 [00:56<03:43,  3.26it/s]

In [ ]:
# net = EffNet(ch_out=13).to(device)

# optimizer = optim.AdamW(net.parameters(), lr=1e-4)
# TrainClassifer(model=net,trn_dl=train_dl,val_dl=val_dl,optimizer=optimizer, 
#                scheduler=None, n_eopchs=30, device=device)

In [ ]:
# loss_val = 0.0
# correct_val = 0.0

# net = EffNet(ch_out=13).to(device)
# net.load_state_dict(torch.load('EffNet3D/test_normal_13class_posweight_bestmetric_correct.pt'))
# loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([1, 30, 1, 15, 1, 30, 30, 1, 18, 18, 1, 18, 18]).to(device))

# net.eval()
        
# for _, (imgs, labels) in tqdm(enumerate(val_dl)):
#     imgs = imgs.to(device)
#     labels = labels.to(device)
#     with torch.no_grad():
#         outputs = net(imgs)
#         #outputs = model(imgs.unsqueeze(1))
#         loss = loss_fn(outputs, labels)
#         loss_val += loss.item()
# print(loss_val / len(val_dl))